# Assigment 2

In [5]:
# install.packages("tidyverse")

In [1]:
library('tidyverse')

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.0
v ggplot2   3.5.1     v tibble    3.2.1
v lubridate 1.9.3     v tidyr     1.3.1
v purrr     1.0.1     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


# 1. Loading and processing the data

### 1. Load the data, either from the url or from the path provided

In [2]:
data <- read_csv('/Users/gabriel/Documents/GitHub/CausalAI-Course/data/wage2015_subsample_inference.csv')
data <- select(data, -rownames)
dim(data)

Rows: 5150 Columns: 21
-- Column specification --------------------------------------------------------
Delimiter: ","
dbl (21): rownames, wage, lwage, sex, shs, hsg, scl, clg, ad, mw, so, we, ne...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 5150   20

### 2. As in Group Assignment 1 2024 - 2 #1044 , generate the extra-flexible model. This means that it contains all two-way interactions between the experience polynomials and the indicator variables

In [3]:
# Convertir las variables occ2 e ind2 a factores
data$occ2 <- factor(data$occ2)
data$ind2 <- factor(data$ind2)

In [4]:
X_extra_flexible <- model.matrix(~ sex + (exp1 + exp2 + exp3 + exp4 + hsg + scl + clg + ad + so + we + ne + C(occ2) + C(ind2)) ^ 2, data = data)

#### 2.1. Generate the array for the outcome variable Y and normalize it

In [5]:
Y <- data$lwage
Y <- (Y-mean(Y))/ sd(Y)
Y <- as.vector(Y)
length(Y)

[1] 5150

#### 2.2. Generate the array for the predictors X (do not generate an intercept) and normalize its colums.

In [6]:
library(dplyr)

In [7]:
X <- model.matrix(~ 0 + sex + (exp1 + exp2 + exp3 + exp4 +
                    hsg + scl + clg + ad + so + we + ne + C(occ2) + C(ind2))^2, data = data)
X <- as.data.frame(X)

vars_norm <- c("exp1", "exp2", "exp3", "exp4")

for (var in vars_norm) {
  X[[var]] <- (X[[var]] - mean(X[[var]])) / sd(X[[var]])
}
X <- as.matrix(X)
dim(X)

[1] 5150  980

### 3. Split between training and testing samples. The testing sample should be 10% of the total.

In [27]:
data <- cbind(Y, X)

set.seed(123)

data <- as.data.frame(data)
train_index <- sample(1:nrow(data), size = 0.9 * nrow(data))
train_sample <- data[train_index, ]  # Datos de entrenamiento
test_sample <- data[-train_index, ] 

dim(train_sample)
dim(test_sample)

[1] 4635  981

[1] 515 981

# 2. Creating the Lasso Cross-Validation Procedure

In [9]:
library(glmnet) 
library(caret)

Loading required package: Matrix


Attaching package: 'Matrix'


The following objects are masked from 'package:tidyr':

    expand, pack, unpack


Loaded glmnet 4.1-8

Loading required package: lattice


Attaching package: 'caret'


The following object is masked from 'package:purrr':

    lift




### 4. Program a function that generates a logarithmically spaced grid. The input arguments should be the lower and upper bounds of the grid, as well as the natural logarithm of the spacing between each element of the grid. The output should be the logarithmically spaced grid, meaning that if we take the natural logarithm of each entry in the grid, they will be equally spaced. This will be the grid of values for λ values to try during cross-validation.

In [10]:
log_grid <- function(lower_bound, upper_bound, niter) {
  exp(seq(log(lower_bound), log(upper_bound), length.out = niter))
}

### 5. Program a function to generate k folds. It should take as input the array to be split rowwise and the number of folds desired. It should output a list of k 1d arrays of booleans; these arrays should all be the same length as the number of rows in the input array, and when they are all summed together they should add up to an array of all true values. Create your own procedure for splitting. You can aid yourself with third party packages like numpy in Python or Statistics in Julia, but do not use a pre-programmed third party splitting procedure like sk-learns's KFolds in Python.

In [11]:
generate_folds <- function(Y, k) {
  n <- length(Y)
  fold_indices <- sample(rep(1:k, length.out = n))
  folds <- lapply(1:k, function(i) {
    fold <- fold_indices == i
    return(fold)
  })
  return(folds)
}

### 6. Program a function that integrates those that you programmed in the last two items to find the value of λ that minimizes the testing mean square error across folds. It should take the following inputs:
- Y: an array for the outcome variable.
- X: an array of predictors.
- lambda_bounds: the lower and upper bounds of the grid of lambda values.
- k: number of folds

### The output should be a dictionary (a list in R) with the following entries:
- optimal_lambda: The lambda that minimizes the testing MSE across folds.
- optimal_coef: An array with the coefficients found for the optimal lambda
- all_lambdas: The grid of lambdas.
- all_mse: An array with the testing MSE across folds for each lambda.

### The procedure goes as follows: With each lambda in the grid -> For each split -> Train and test a lasso model.

With this, you can get the testing MSE associated with the value of lambda in each iteration. Feel free to use a third party Lasso estimator, as long as it is not one that calculates the optimal lambda value. Instead, the third party estimator you use should only estimate the regression with the penalty weight provided so you can get the testing MSE with that specific weight.

In [12]:
optimize_lambda <- function(Y, X, lambda_bounds, k) {
  lambdas <- log_grid(lambda_bounds[1], lambda_bounds[2], niter)
  folds <- generate_folds(Y, k)
  avg_mse <- numeric(length(lambdas))

  for (i in seq_along(lambdas)) {
    lambda <- lambdas[i]
    fold_pe <- numeric(k)

    for (j in seq_along(folds)) {
      fold <- folds[[j]]
      X_train <- X[!fold, ]
      Y_train <- Y[!fold]
      X_test <- X[fold, ]
      Y_test <- Y[fold]

      model <- glmnet(X_train, Y_train, alpha = 1, lambda = lambda)
      Y_pred <- predict(model, newx = X_test)
      fold_pe[j] <- sum((Y_test - Y_pred)^2)
    }

    avg_mse[i] <- mean(fold_pe)
  }

  optimal_lambda <- lambdas[which.min(avg_mse)]
  optimal_model <- glmnet(X, Y, alpha = 1, lambda = optimal_lambda)

  return(list(optimal_lambda = optimal_lambda,
              optimal_coef = coef(optimal_model),
              model = optimal_model,  # Guardar el modelo aquí
              all_lambdas = lambdas,
              all_mse = avg_mse))
}

### 7. Program a function for predicting the outcome variable through model estimated with the optimal lambda. It should take as inputs
- optimal_model: A dictionary with the values outputed by the function defined for the previous point.
- X: an array of predictors.

### The output should be an array of predicted values.

In [13]:
predict_outcome <- function(optimal_model, X) {
  
  lambda_min <- optimal_model$optimal_lambda
  predictions <- predict(optimal_model$model, newx = X, s = lambda_min)
  
  return(predictions)
}

Probando las funciones

# 3. Applying the Lasso Cross-Validation Procedure

### 8. Fit a simple OLS model with the training sample.

In [41]:
model_OLS <- lm(Y~., data = train_sample)

### 9. Find the optimal lambda and its related coefficients with the function programmed in 3. and the training sample. Print the lambda and the coefficients.

In [63]:
# Generando los inputs requeridos para que las funciones corran
k <- 5
lambda_bounds <- c(0.001, 100)
niter = 100

X_train <- train_sample[, -1]
X_train <- as.matrix(X_train)

model_opt <- optimize_lambda(train_sample$Y, X_train, lambda_bounds, k)
print(model_opt$optimal_lambda)
print(model_opt$optimal_coef)

### 10. Now use each language's hdm package to fit a Lasso model with the theoretical optimal lambda value

In [36]:
fit_rlasso <- hdm::rlasso(Y ~ X, post = FALSE) 
fit_rlasso$lambda0
fit_rlasso$iter

[1] 314.2306

[1] 9

### 11. Report the testing MSE and $R^2$ for the OLS model and, the cross-validation lambda Lasso model, and the hdm theoretical lambda model. Use the function programmed in 4. to compute the predictions of the cross-validation Lasso model.

In [65]:
# Compute the Out-Of-Sample Performance of OLS
Y_hat_OLS <- predict(model_OLS, newdata = test_sample)
mse_OLS <- mean((test_sample$Y - Y_hat_OLS^2))
R_sq_OLS <- 1 - mse_OLS / mean ((test_sample$Y - mean(test_sample$Y))^2)

print(paste0("MSE of OLS model is; ", mse_OLS))
print(paste0("R square of OLS model is; ", R_sq_OLS))

Warning message in predict.lm(model_OLS, newdata = test_sample):
"prediction from a rank-deficient fit may be misleading"


[1] "MSE of OLS model is; -4.18676720919766"
[1] "R square of OLS model is; 5.30772979469966"


In [77]:
# Compute the Out-Of-Sample Performance of Cross validation
model_CV <- glmnet(X_train, train_sample$Y, alpha = 1, lambda = model_opt$optimal_lambda)
X_test <- test_sample[, -1] 
Y_hat_CV <- predict(model_CV, newx = as.matrix(X_test))
mse_CV <- mean((test_sample$Y - Y_hat_CV^2))
R_sq_CV <- 1 - mse_CV / mean ((test_sample$Y - mean(test_sample$Y))^2)

print(paste0("MSE of CV procedure model is; ", mse_CV))
print(paste0("R square of CV procedure model is; ", R_sq_CV))

[1] "MSE of CV procedure model is; -0.335718491311098"
[1] "R square of CV procedure model is; 1.3454179502685"


In [78]:
# Compute the Out-Of-Sample Performance of HDM package
model_HDM <- glmnet(X_train, train_sample$Y, alpha = 1, lambda = fit_rlasso$lambda0)
Y_hat_HDM <- predict(model_HDM, newx = as.matrix(X_test))
mse_HDM <- mean((test_sample$Y - Y_hat_HDM^2))
R_sq_HDM <- 1 - mse_HDM / mean ((test_sample$Y - mean(test_sample$Y))^2)

print(paste0("MSE of HDM package model is; ", mse_HDM))
print(paste0("R square of HDM package model is; ", R_sq_HDM))

[1] "MSE of HDM package model is; -0.0393337311592602"
[1] "R square of HDM package model is; 1.04047014729628"
